In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from pandas import read_csv

In [ ]:
#load dataset
url = 'C:\\Users\\user\\Desktop\\ML\\PROJECT\\CTR_SampleData'
df= pd.read_csv(url, header=0, sep='\t')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#checking for null/missing data
df.info()

In [ ]:
#checking varieties in each feature
df.apply(pd.Series.nunique)

In [ ]:
sb.countplot(x='click',data=df, palette='hls')
plt.show();

In [ ]:
plt.figure(figsize=(20,10))
c= df.corr()
sb.heatmap(c)
c

In [ ]:
#changing 'hour' format
df['hour']=pd.to_datetime(df['hour'],format='%y%m%d%H')
df['dayOfMonth']=df['hour'].dt.day
df['hou']=df['hour'].dt.hour
df=df.drop(['hour'],axis=1)
df = df.rename(columns={'hou': 'hour'})

In [ ]:
df=df.drop(['id'],axis=1)

In [ ]:
def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self
df = convert_obj_to_int(df)

In [ ]:
y=df.click
X=df.drop('click',axis=1)
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

### Null data

https://www.kaggle.com/c/avazu-ctr-prediction/discussion/10819#58338

NULLs for siteid, sitedomain, site_category: 85f751fd, c4e18dd6, 50e219e0

NULLs for appid, appdomain, app_category: ecad2386, 7801e8d9, 07d7df22

In [ ]:
si=df.loc[df.site_id == '85f751fd', 'site_id'].count()
si/1000000*100

In [ ]:
sdm=df.loc[df.site_domain== 'c4e18dd6', 'site_domain'].count()
sdm/1000000*100

In [ ]:
sc=df.loc[df.site_category == '50e219e0', 'site_category'].count()
sc/1000000*100

In [ ]:
api=df.loc[df.app_id == 'ecad2386', 'app_id'].count()
api/1000000*100

In [ ]:
apdm=df.loc[df.app_domain== '7801e8d9', 'app_domain'].count()
apdm/1000000*100

In [ ]:
apc=df.loc[df.app_category == '07d7df22', 'app_category'].count()
apc/1000000*100

In [ ]:
from numpy import nan
# replace '-2' values with null values
df[[site_id]] = df[[site_id]].replace(-2, 85f751fd)
# count the number of nan values in each column
print(df.isnull().sum())

In [ ]:
# evaluate model on data where missing values cause errors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# define the model
model = LinearDiscriminantAnalysis()
# define the model evaluation procedure
cv = KFold(n_splits=3, shuffle=True, random_state=1)
# evaluate the model
result = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
# report the mean performance
print('Accuracy: %.3f' % result.mean())

In [ ]:
# drop rows with missing values
df.dropna(inplace=True)
# summarize the shape of the data with missing rows removed
print(df.shape)

In [ ]:
# evaluate model on data after removed missing data
model = LinearDiscriminantAnalysis()
# define the model evaluation procedure
cv = KFold(n_splits=3, shuffle=True, random_state=1)
# evaluate the model
result = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
# report the mean performance
print('Accuracy: %.3f' % result.mean())

### Select Features

In [ ]:
# report which features were selected by RFE
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=10)

rfe.fit(X, y)
# summarize all features
for i in range(X.shape[1]):
	print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# feature extraction
model = LogisticRegression(solver='lbfgs')
rfe = RFE(model, 3)
fit = rfe.fit(x, y)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)

**Feature Importance with XGBoost**

In [ ]:
# plot feature importance using built-in function

from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

model = XGBClassifier()
model.fit(X, y)
# plot feature importance
plot_importance(model)
pyplot.show()

**Feature Importance with RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = df_X.columns
rnd_clf = RandomForestClassifier( n_jobs=-1, random_state=0)
rnd_clf.fit(df_X, df_y)
importances = rnd_clf.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(13,13))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='skyblue', align='center',height=0.9)
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

### Outliners

In [ ]:
#Z-score
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(X))
df_o=X
df_o =df_o[(z < 3).all(axis=1)]

df_o.shape

lst1=df_o.index.values.tolist()

In [ ]:
Q1 = x_train.quantile(0.25)
Q3 = x_train.quantile(0.75)
IQR = Q3 - Q1
df_o2 = X[~((X < (Q1 - 1.5 * IQR)) |(X > (Q3 + 1.5 * IQR))).any(axis=1)]

df_o2.shape

lst2=df_o2.index.values.tolist()

In [ ]:
from sklearn.ensemble import IsolationForest
from collections import Counter

clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(.12), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

clf.fit(X)
isof_outliers = clf.predict(X)
iso=isof_outliers.tolist()
Counter(iso)
listOfTrainIndexes=X.index.tolist()

lst3=[]
for i in range(len(iso)):
    if iso[i]==1:
        lst3.append(listOfTrainIndexes[i])
        

In [ ]:
resultLst=np.unique(lst1+lst2+lst3)
len(resultLst) 

### Data transform

**ordinal encode/One-hot encode**

In [ ]:
print('Input', X.shape)
print('Output', y.shape)

In [ ]:
# ordinal encode
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

# ordinal encode input variables
ordinal_encoder = OrdinalEncoder()
X_ore = ordinal_encoder.fit_transform(X)
# ordinal encode target variable
label_encoder = LabelEncoder()
y_ore = label_encoder.fit_transform(y)
# summarize the transformed data
print('Input', X_ore.shape)
print(X_ore[:5, :])
print('Output', y_ore.shape)
print(y_ore[:5])

In [ ]:
# evaluate logistic regression with an ordinal encoding
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score

X,y=X_ore,y_ore
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

# ordinal encode input variables
ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit(X_train)
X_train = ordinal_encoder.transform(X_train)
X_test = ordinal_encoder.transform(X_test)
# ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
# define the model
model = LogisticRegression()
# fit on the training set
model.fit(X_train, y_train)
# predict on test set
yhat = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.2f' % (accuracy*100))


In [ ]:
# one-hot encode
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# one hot encode input variables
onehot_encoder = OneHotEncoder(sparse=False)
X_ohe = onehot_encoder.fit_transform(X)
# ordinal encode target variable
label_encoder = LabelEncoder()
y_ohe = label_encoder.fit_transform(y)
# summarize the transformed data
print('Input', X_ohe.shape)
print(X_ohe[:5, :])

In [ ]:
# evaluate logistic regression with an one-hot encoding
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

X,y=X_ohe,y_ohe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# one-hot encode input variables
onehot_encoder = OneHotEncoder()
onehot_encoder.fit(X_train)
X_train = onehot_encoder.transform(X_train)
X_test = onehot_encoder.transform(X_test)
# ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
# define the model
model = LogisticRegression()
# fit on the training set
model.fit(X_train, y_train)
# predict on test set
yhat = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.2f' % (accuracy*100))

### Dimensionality Reduction

**PCA** (Principal Component Analysis), might be the most popular technique for dimensionality reduction with dense data (few zero values).

In [ ]:
# evaluate pca with logistic regression algorithm for classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

# define the pipeline
steps = [('pca', PCA(n_components=10)), ('m', LogisticRegression())]
model = Pipeline(steps=steps)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

**SVD** (Singular Value Decomposition), is one of the most popular techniques for dimensionality reduction for sparse data (data with many zero values).

In [ ]:
# evaluate svd with logistic regression algorithm for classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

# define the pipeline
steps = [('svd', TruncatedSVD(n_components=10)), ('m', LogisticRegression())]
model = Pipeline(steps=steps)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

**LDA** (Linear Discriminant Analysis), is a multi-class classification algorithm that can be used for dimensionality reduction.

The number of dimensions for the projection is limited to 1 and C-1, where C is the number of classes. In this case, our dataset is a binary classification problem (two classes), limiting the number of dimensions to 1.

In [ ]:
# evaluate lda with logistic regression algorithm for classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

X = x_train
y = y_train
# define the pipeline
steps = [('lda', LinearDiscriminantAnalysis(n_components=1)), ('m', LogisticRegression())]
model = Pipeline(steps=steps)
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

### Imbalancing data

In [ ]:
# Oversampling imbalanced training data with SMOTE
from collections import Counter
from imblearn.over_sampling import SMOTE

X=X
y=y
# count class(click 0 and 1) dist.
counter = Counter(y)
print(counter)

#  new class dist
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

counter = Counter(y)
print(counter)


In [ ]:
# Oversampling imbalanced training data with SMOTE and random undersample
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

X=X
y=y

counter = Counter(y)
print(counter)

over_samp = SMOTE(sampling_strategy=0.3)
under_samp = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over_samp), ('u', under_samp)]
pipeline = Pipeline(steps=steps)

X, y = pipeline.fit_resample(X, y)

counter = Counter(y)
print(counter)

In [ ]:
# borderline-SMOTE with SVM 
from collections import Counter
from imblearn.over_sampling import SVMSMOTE

X=X
y=y
counter = Counter(y)
print(counter)

oversample = SVMSMOTE()
X, y = oversample.fit_resample(X, y)

counter = Counter(y)
print(counter)

### ROC vs. Precision-Recall Curves

- **ROC curves** should be used when there are roughly equal numbers of observations for each class.
- **Precision-Recall curves** should be used when there is a moderate to large class imbalance.

In [ ]:
# roc curve and auc on an imbalanced dataset
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate 2 class dataset
X, y =X_ore, y_ore
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(testy))]
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)
lr_auc = roc_auc_score(testy, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
# precision-recall curve and f1 for an imbalanced dataset
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot
# generate 2 class dataset
X, y = X_ore, y_ore
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# predict class values
yhat = model.predict(testX)
# calculate precision and recall for each threshold
lr_precision, lr_recall, _ = precision_recall_curve(testy, lr_probs)
# calculate scores
lr_f1, lr_auc = f1_score(testy, yhat), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
# plot the precision-recall curves
no_skill = len(testy[testy==1]) / len(testy)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()